In [ ]:
 !pip -q install requests beautifulsoup4 pandas lxml dateparser langdetect tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import json
import random
import hashlib
from urllib.parse import urljoin, urlparse

import requests
import pandas as pd
# import feedparser
from bs4 import BeautifulSoup
from langdetect import detect
from tqdm import tqdm


# trial last

In [ ]:
import re, json, os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import xml.etree.ElementTree as ET
import dateparser
from datetime import datetime, timezone

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/122.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9,bn;q=0.8",
    "Connection": "keep-alive",
}


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def debug_url(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=20, allow_redirects=True)
        ct = r.headers.get("Content-Type", "")
        print("\nURL:", url)
        print("Final:", r.url)
        print("Status:", r.status_code)
        print("Content-Type:", ct)
        print("First 300 chars:\n", (r.text[:300] if r.text else ""))
    except Exception as e:
        print("\nURL:", url)
        print("ERROR:", repr(e))

test_urls = [
    "https://www.thedailystar.net/rss.xml",
    "https://www.dhakatribune.com/topic/rss/feed",
    "https://www.prothomalo.com/feed",
    "https://banglatribune.com/feed/",
    "https://www.kalerkantho.com/rss.xml",
]

for u in test_urls:
    debug_url(u)



URL: https://www.thedailystar.net/rss.xml
Final: https://www.thedailystar.net/rss.xml
Status: 200
Content-Type: application/rss+xml; charset=utf-8
First 300 chars:
 <?xml version="1.0" encoding="utf-8"?>
<rss xmlns:dc="http://purl.org/dc/elements/1.1/" version="2.0" xml:base="https://www.thedailystar.net/">
  <channel>
    <title>The Daily Star - Journalism without fear or favour</title>
    <link>https://www.thedailystar.net/</link>
    <description><![CDATA[]

URL: https://www.dhakatribune.com/topic/rss/feed
Final: https://www.dhakatribune.com/topic/rss/feed
Status: 200
Content-Type: text/html; charset=utf-8
First 300 chars:
 <!DOCTYPE html><html lang="en" ><head prefix="og: http://ogp.me/ns#"><!--turn off IE compatibility mode --><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta charset="utf-8"><!--<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>--><meta name="viewport" content="width=devi

URL: https://www.prothomalo.com/feed
Final: https://prod-qt-i

In [ ]:
def get_sitemaps_from_robots(site_base):
    robots_url = site_base.rstrip("/") + "/robots.txt"
    try:
        r = requests.get(robots_url, headers=HEADERS, timeout=20)
        r.raise_for_status()
        sitemaps = re.findall(r"(?i)^sitemap:\s*(\S+)", r.text, flags=re.MULTILINE)
        return list(dict.fromkeys(sitemaps))  # dedup keep order
    except Exception as e:
        print("robots fail:", robots_url, repr(e))
        return []


In [ ]:
def fetch_text(url):
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text

def parse_sitemap(url, limit=50000):
    """
    Returns:
      urls: list of page URLs
      subs: list of child sitemap URLs (if sitemapindex)
    """
    xml = fetch_text(url)
    root = ET.fromstring(xml)

    ns = {"sm": "http://www.sitemaps.org/schemas/sitemap/0.9"}
    urls, subs = [], []

    if root.tag.endswith("sitemapindex"):
        for sm in root.findall("sm:sitemap", ns):
            loc = sm.find("sm:loc", ns)
            if loc is not None and loc.text:
                subs.append(loc.text.strip())

    elif root.tag.endswith("urlset"):
        for u in root.findall("sm:url", ns):
            loc = u.find("sm:loc", ns)
            if loc is not None and loc.text:
                urls.append(loc.text.strip())
                if len(urls) >= limit:
                    break

    return urls, subs

def collect_urls_from_sitemaps(sitemap_urls, max_urls=20000):
    collected = []
    queue = list(sitemap_urls)
    seen_sm = set()

    while queue and len(collected) < max_urls:
        sm = queue.pop(0)
        if sm in seen_sm:
            continue
        seen_sm.add(sm)

        try:
            urls, subs = parse_sitemap(sm)
            if subs:
                queue.extend(subs)
            if urls:
                collected.extend(urls)
                collected = list(dict.fromkeys(collected))  # dedup keep order
        except Exception as e:
            print("sitemap parse failed:", sm, repr(e))

    return collected[:max_urls]


In [ ]:
english_sites = [
    "https://www.thedailystar.net",
    "https://www.newagebd.net",
    "https://www.daily-sun.com",
    "https://www.dhakatribune.com",
    "https://www.dailynewnation.com",
]

bangla_sites = [
    "https://www.prothomalo.com",
    "https://bangla.bdnews24.com",
    "https://www.kalerkantho.com",
    "https://banglatribune.com",
    "https://www.dhakapost.com",
]


In [ ]:
def url_pool_for_site(site_base, max_urls=15000):
    sitemaps = get_sitemaps_from_robots(site_base)
    if not sitemaps:
        print("No sitemaps found for:", site_base)
        return []
    urls = collect_urls_from_sitemaps(sitemaps, max_urls=max_urls)
    print("Collected", len(urls), "urls from", site_base)
    return urls

en_urls, bn_urls = [], []

for s in english_sites:
    en_urls += url_pool_for_site(s, max_urls=20000)

for s in bangla_sites:
    bn_urls += url_pool_for_site(s, max_urls=20000)

en_urls = list(dict.fromkeys(en_urls))
bn_urls = list(dict.fromkeys(bn_urls))

print("EN URL pool:", len(en_urls))
print("BN URL pool:", len(bn_urls))


No sitemaps found for: https://www.thedailystar.net
No sitemaps found for: https://www.newagebd.net
sitemap parse failed: https://www.daily-sun.com/sitemap.xml HTTPError('403 Client Error: Forbidden for url: https://www.daily-sun.com/sitemap.xml')
sitemap parse failed: https://www.daily-sun.com/news_sitemap.xml HTTPError('403 Client Error: Forbidden for url: https://www.daily-sun.com/news_sitemap.xml')
Collected 0 urls from https://www.daily-sun.com
Collected 20000 urls from https://www.dhakatribune.com
Collected 20000 urls from https://www.dailynewnation.com
Collected 20000 urls from https://www.prothomalo.com
robots fail: https://bangla.bdnews24.com/robots.txt HTTPError('403 Client Error: Forbidden for url: https://bangla.bdnews24.com/robots.txt')
No sitemaps found for: https://bangla.bdnews24.com
sitemap parse failed: https://www.kalerkantho.com/daily-sitemap/sitemap-section.xml HTTPError('403 Client Error: Forbidden for url: https://www.kalerkantho.com/daily-sitemap/sitemap-section

In [ ]:
def looks_like_article(url):
    bad = ["#", "/tag/", "/topic/", "/author/", "/video", "/photo", "/gallery", "/search"]
    if any(b in url for b in bad):
        return False
    path = urlparse(url).path.strip("/")
    if path == "":
        return False
    return len(path.split("/")) >= 2

en_urls_filtered = [u for u in en_urls if looks_like_article(u)]
bn_urls_filtered = [u for u in bn_urls if looks_like_article(u)]

print("Filtered EN:", len(en_urls_filtered))
print("Filtered BN:", len(bn_urls_filtered))


Filtered EN: 38992
Filtered BN: 36953


In [ ]:
DATE_META_KEYS = [
    ("meta", {"property": "article:published_time"}),
    ("meta", {"property": "article:modified_time"}),
    ("meta", {"name": "pubdate"}),
    ("meta", {"name": "publishdate"}),
    ("meta", {"name": "timestamp"}),
    ("meta", {"name": "DC.date.issued"}),
    ("meta", {"name": "DC.Date"}),
    ("meta", {"name": "date"}),
    ("meta", {"name": "Date"}),
    ("meta", {"name": "article.published"}),
    ("meta", {"name": "article:published_time"}),
    ("meta", {"name": "parsely-pub-date"}),
    ("meta", {"itemprop": "datePublished"}),
    ("meta", {"itemprop": "dateModified"}),
]

def _normalize_dt(dt: datetime):
    if not dt:
        return None
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc).isoformat()

def _parse_date_str(s: str):
    if not s:
        return None
    s = s.strip()
    return dateparser.parse(
        s,
        settings={
            "RETURN_AS_TIMEZONE_AWARE": True,
            "TO_TIMEZONE": "UTC",
            "PREFER_DATES_FROM": "past",
            "STRICT_PARSING": False,
        },
        languages=["en", "bn"],
    )

def extract_date_from_jsonld(soup: BeautifulSoup):
    for tag in soup.find_all("script", type="application/ld+json"):
        txt = tag.string or tag.get_text(strip=True)
        if not txt:
            continue
        txt = txt.strip()
        try:
            data = json.loads(txt)
        except Exception:
            m = re.search(r"(\{.*\}|\[.*\])", txt, flags=re.DOTALL)
            if not m:
                continue
            try:
                data = json.loads(m.group(1))
            except Exception:
                continue

        candidates = []
        stack = data if isinstance(data, list) else [data]

        for obj in stack:
            if not isinstance(obj, dict):
                continue
            if "@graph" in obj and isinstance(obj["@graph"], list):
                stack.extend([x for x in obj["@graph"] if isinstance(x, dict)])

            for k in ("datePublished", "dateCreated", "dateModified", "uploadDate"):
                v = obj.get(k)
                if isinstance(v, str):
                    candidates.append(v)

        for c in candidates:
            dt = _parse_date_str(c)
            if dt:
                return dt
    return None

def extract_date_from_meta_and_time(soup: BeautifulSoup):
    for tag_name, attrs in DATE_META_KEYS:
        el = soup.find(tag_name, attrs=attrs)
        if el:
            v = el.get("content") or el.get("value") or el.get_text(strip=True)
            dt = _parse_date_str(v)
            if dt:
                return dt

    t = soup.find("time")
    if t:
        v = t.get("datetime") or t.get_text(strip=True)
        dt = _parse_date_str(v)
        if dt:
            return dt

    return None

def extract_date_from_url(url: str):
    path = urlparse(url).path
    m = re.search(r"(20\d{2})[/-](\d{1,2})[/-](\d{1,2})", path)
    if m:
        y, mo, d = map(int, m.groups())
        try:
            return datetime(y, mo, d, tzinfo=timezone.utc)
        except Exception:
            return None

    m2 = re.search(r"(20\d{2})[/-](\d{1,2})", path)
    if m2:
        y, mo = map(int, m2.groups())
        try:
            return datetime(y, mo, 1, tzinfo=timezone.utc)
        except Exception:
            return None
    return None

def extract_article_date(html: str, url: str):
    if not html:
        return None, "missing"

    soup = BeautifulSoup(html, "lxml")

    dt = extract_date_from_jsonld(soup)
    if dt:
        return _normalize_dt(dt), "jsonld"

    dt = extract_date_from_meta_and_time(soup)
    if dt:
        return _normalize_dt(dt), "meta_or_time"

    dt = extract_date_from_url(url)
    if dt:
        return _normalize_dt(dt), "url"

    return None, "missing"


In [ ]:
def extract_title_and_body(html: str):
    soup = BeautifulSoup(html, "lxml")

    # title
    title = ""
    ogt = soup.find("meta", property="og:title")
    if ogt and ogt.get("content"):
        title = ogt["content"].strip()
    elif soup.title and soup.title.get_text(strip=True):
        title = soup.title.get_text(strip=True)

    # body: collect <p> inside <article> if present, else all <p>
    body_parts = []
    article = soup.find("article")
    if article:
        ps = article.find_all("p")
    else:
        ps = soup.find_all("p")

    for p in ps:
        txt = p.get_text(" ", strip=True)
        if txt and len(txt) >= 30:
            body_parts.append(txt)

    body = "\n".join(body_parts).strip()
    return title, body


In [ ]:
# -----------------------------
# 1) SOURCE MAP (ONLY THESE!)
# -----------------------------
SOURCE_MAP = {
    "thedailystar.net": "TheDailyStar",
    "dhakatribune.com": "DhakaTribune",
    "newagebd.net": "NewAgeBD",
    "daily-sun.com": "DailySun",
    "dailynewnation.com": "DailyNewNation",

    "prothomalo.com": "ProthomAlo",
    "banglatribune.com": "BanglaTribune",
    "kalerkantho.com": "KalerKantho",
    "bangla.bdnews24.com": "BDNews24Bangla",
    "dhakapost.com": "DhakaPost",
}

DOMAIN_LANG = {
    "thedailystar.net": "en",
    "dhakatribune.com": "en",
    "newagebd.net": "en",
    "daily-sun.com": "en",
    "dailynewnation.com": "en",

    "prothomalo.com": "bn",
    "banglatribune.com": "bn",
    "kalerkantho.com": "bn",
    "bangla.bdnews24.com": "bn",
    "dhakapost.com": "bn",
}


In [ ]:
from urllib.parse import urlparse

def get_domain(url):
    netloc = urlparse(url).netloc.lower()
    return netloc.replace("www.", "")

def map_source_and_lang(url):
    domain = get_domain(url)
    source = SOURCE_MAP.get(domain, "UNKNOWN")
    language = DOMAIN_LANG.get(domain, "unknown")
    return source, language


In [ ]:
def crawl_urls(urls, forced_lang="en", source_name="SITEMAP", max_articles=100):
    rows = []
    for url in urls:
        try:
            r = requests.get(url, headers=HEADERS, timeout=25, allow_redirects=True)
            if r.status_code != 200:
                continue
            html = r.text
        except Exception:
            continue

        title, body = extract_title_and_body(html)

        # skip low-content pages
        if not body or len(body.split()) < 80:
            continue

        iso_date, date_method = extract_article_date(html, url)

        rows.append({
            "title": title,
            "body": body,
            "url": url,
            "date": iso_date,
            "date_method": date_method,
            "language": forced_lang,
            "tokens": len(body.split()),
            "source": source_name,
        })

        if len(rows) >= max_articles:
            break

    return pd.DataFrame(rows)


In [ ]:
def crawl_urls(urls, forced_lang="en", max_articles=100):
    rows = []

    for url in urls:
        try:
            r = requests.get(url, headers=HEADERS, timeout=25, allow_redirects=True)
            if r.status_code != 200:
                continue
            html = r.text
        except Exception:
            continue

        title, body = extract_title_and_body(html)

        # Skip weak pages
        if not body or len(body.split()) < 80:
            continue

        iso_date, date_method = extract_article_date(html, url)

        source, language = map_source_and_lang(url)

        # Skip if domain not in your approved list
        if source == "UNKNOWN" or language == "unknown":
            continue

        rows.append({
            "title": title,
            "body": body,
            "url": url,
            "date": iso_date,
            "date_method": date_method,
            "language": language,   # ← mapped
            "tokens": len(body.split()),
            "source": source,       # ← mapped
        })

        if len(rows) >= max_articles:
            break

    return pd.DataFrame(rows)


In [ ]:
english_df = crawl_urls(en_urls_filtered, forced_lang="en", max_articles=3000)
bangla_df  = crawl_urls(bn_urls_filtered, forced_lang="bn", max_articles=3000)

print("Crawled EN:", len(english_df))
print("Crawled BN:", len(bangla_df))

# english_df.head(2)



Crawled EN: 3000
Crawled BN: 3000


In [ ]:
# bangla_df.head(5)

,title,body,url,date,date_method,language,tokens,source
0,"মিরপুরে বিএনপি-জামায়াত সংঘর্ষ, বেশ কয়েকজন আহত",রাজধানীর মিরপুরের পীরেরবাগে স্থানীয় বিএনপি ও জ...,https://www.prothomalo.com/politics/pacz4f0h3u,2026-01-20T20:07:16+00:00,jsonld,bn,154,ProthomAlo
1,আশুলিয়ায় লাশ পোড়ানোর মামলায় যুক্তিতর্ক সম্পন্ন...,জুলাই গণ–অভ্যুত্থানের সময় আশুলিয়ায় ছয়জনের লাশ ...,https://www.prothomalo.com/bangladesh/sy1fo9rde7,2026-01-20T19:50:44+00:00,jsonld,bn,137,ProthomAlo
2,আবেদ আলীর ছেলে সোহানুর রহমান গ্রেপ্তার,প্রশ্নপত্র ফাঁসকাণ্ডে আলোচিত সাবেক পিএসসি চেয়া...,https://www.prothomalo.com/bangladesh/cas5b5sx4t,2026-01-20T19:46:32+00:00,jsonld,bn,243,ProthomAlo
3,বাংলাদেশ থেকে কূটনীতিকদের পরিবারের সদস্যদের ফি...,বাংলাদেশে ভারতের হাইকমিশন ও সহকারী হাইকমিশনগুল...,https://www.prothomalo.com/bangladesh/yjtqn8ais3,2026-01-20T19:40:32+00:00,jsonld,bn,141,ProthomAlo
4,এবার উপাচার্যের পদত্যাগ দাবি এশিয়া প্যাসিফিকের...,ঢাকার বেসরকারি বিশ্ববিদ্যালয় ইউনিভার্সিটি অব এ...,https://www.prothomalo.com/bangladesh/jaqan8nqmm,2026-01-20T19:08:54+00:00,jsonld,bn,430,ProthomAlo


In [ ]:
def save_jsonl(df, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        for rec in df.to_dict(orient="records"):
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print("Saved:", out_path)

def save_json_array(df, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df.to_json(out_path, orient="records", force_ascii=False, indent=2)
    print("Saved:", out_path)

# CHANGE this folder as you want
OUT_DIR = "/content/drive/MyDrive/CLIR_dataset"

save_jsonl(english_df, f"{OUT_DIR}/english_news.jsonl")
save_jsonl(bangla_df,  f"{OUT_DIR}/bangla_news.jsonl")

save_json_array(english_df, f"{OUT_DIR}/english_news.json")
save_json_array(bangla_df,  f"{OUT_DIR}/bangla_news.json")


Saved: /content/drive/MyDrive/CLIR_dataset/english_news.jsonl
Saved: /content/drive/MyDrive/CLIR_dataset/bangla_news.jsonl
Saved: /content/drive/MyDrive/CLIR_dataset/english_news.json
Saved: /content/drive/MyDrive/CLIR_dataset/bangla_news.json


In [ ]:
def date_coverage(df, name):
    total = len(df)
    missing = df["date"].isna().sum()
    print(f"{name}: total={total}, date_missing={missing}, date_coverage={(total-missing)/max(total,1):.2%}")
    print(df["date_method"].value_counts(dropna=False))

date_coverage(english_df, "English")
date_coverage(bangla_df, "Bangla")
